In [ ]:
pip install pandas requests aiohttp pyarrow

In [2]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

In [5]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [25]:
import requests
import pandas as pd
import re
import os
import time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from bs4 import BeautifulSoup

EMAIL = "matrosova-05@list.ru"

# Функция для получения id вакансий
def fetch_vacancies_ids(search_text="python", area=1, per_page=50, pages=10):
    ids = []
    base_url = "https://api.hh.ru/vacancies"
    
    headers = {
        'User-Agent': f'MyApp ({EMAIL})'
    }
    
    for page in range(pages):
        params = {
            'text': search_text,
            'area': area,
            'per_page': per_page,
            'page': page
        }
        
        try:
            response = requests.get(base_url, params=params, headers=headers)
            
            if response.status_code == 200:
                data = response.json()
                for item in data['items']:
                    ids.append(item['id'])
                print(f"Страница {page + 1}: получено {len(data['items'])} вакансий")
            else:
                print(f"Ошибка при запросе страницы {page}: {response.status_code}")
                if response.status_code == 403:
                    print("Доступ запрещен")
                print(response.text)
        
        except Exception as e:
            print(f"Исключение при запросе страницы {page}: {str(e)}")
        
        # Задержка м/у запросами
        time.sleep(2)
        
    return ids

# Функция для получения деталей вакансии 
def fetch_vacancy_details(vacancy_id):
    url = f"https://api.hh.ru/vacancies/{vacancy_id}"
    
    headers = {
        'User-Agent': f'MyApp ({EMAIL})'
    }
    
    try:
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            vacancy_data = response.json()
            
            description = parse_description(vacancy_data)
            
            vacancy_info = {
                'id': vacancy_data.get('id'),
                'name': vacancy_data.get('name'),
                'salary': vacancy_data.get('salary'),
                'area': vacancy_data.get('area', {}).get('name') if vacancy_data.get('area') else None,
                'published_at': vacancy_data.get('published_at'),
                'employer': vacancy_data.get('employer', {}).get('name') if vacancy_data.get('employer') else None,
                'experience': vacancy_data.get('experience', {}).get('name') if vacancy_data.get('experience') else None,
                'employment': vacancy_data.get('employment', {}).get('name') if vacancy_data.get('employment') else None,
                'description': description,
                'key_skills': [skill['name'] for skill in vacancy_data.get('key_skills', [])],
                'schedule': vacancy_data.get('schedule', {}).get('name') if vacancy_data.get('schedule') else None,
                'professional_roles': [role['name'] for role in vacancy_data.get('professional_roles', [])],
                'url': vacancy_data.get('alternate_url')
            }
            
            return vacancy_info
        else:
            print(f"Ошибка при запросе вакансии {vacancy_id}: {response.status_code}")
            return None
            
    except Exception as e:
        print(f"Исключение при обработке вакансии {vacancy_id}: {str(e)}")
        return None
    
    finally:
        time.sleep(1)

# Очистка
def parse_description(vacancy_data):
    description = vacancy_data.get('description')
    if not description:
        return ''

    soup = BeautifulSoup(description, 'html.parser')
    clean_text = soup.get_text(separator=' ')  # Убираем html теги
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()  # Убираем лишние пробелы и переносы строк
    return clean_text

# Основная многопоточная функция
def main_threaded():
    print("Начало сбора данных через публичное API")
    
    print("Получение списка вакансий...")
    vacancy_ids = fetch_vacancies_ids(pages=10)
    print(f"Найдено вакансий: {len(vacancy_ids)}")

    if not vacancy_ids:
        print("Не удалось получить список вакансий")
        return pd.DataFrame()

    vacancies_details = []
    print("Получение деталей вакансий...")
    
    with ThreadPoolExecutor(max_workers=2) as executor:
        futures = [executor.submit(fetch_vacancy_details, vid) for vid in vacancy_ids]
        
        for i, future in enumerate(as_completed(futures)):
            result = future.result()
            if result:
                vacancies_details.append(result)
            
            if (i + 1) % 10 == 0:
                print(f"Обработано {i + 1}/{len(vacancy_ids)} вакансий")
    
    df = pd.DataFrame(vacancies_details)
    print(f"Собрано: {len(df)} вакансий")
    
    # Сохраняем
    data_folder = "data"
    if not os.path.exists(data_folder):
        os.makedirs(data_folder)
    
    csv_filename = os.path.join(data_folder, f"hh_vacancies_{datetime.now().strftime('%Y-%m-%d_%H-%M')}.csv")
    df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    print(f"Датасет сохранен в {csv_filename}")
    
    return df

if __name__ == "__main__":
    df = main_threaded()

Начало сбора данных через публичное API
Получение списка вакансий...
Страница 1: получено 50 вакансий
Страница 2: получено 50 вакансий
Страница 3: получено 50 вакансий
Страница 4: получено 50 вакансий
Страница 5: получено 50 вакансий
Страница 6: получено 50 вакансий
Страница 7: получено 50 вакансий
Страница 8: получено 50 вакансий
Страница 9: получено 50 вакансий
Страница 10: получено 50 вакансий
Найдено вакансий: 500
Получение деталей вакансий...
Обработано 10/500 вакансий
Обработано 20/500 вакансий
Обработано 30/500 вакансий
Обработано 40/500 вакансий
Обработано 50/500 вакансий
Обработано 60/500 вакансий
Обработано 70/500 вакансий
Обработано 80/500 вакансий
Обработано 90/500 вакансий
Обработано 100/500 вакансий
Обработано 110/500 вакансий
Обработано 120/500 вакансий
Обработано 130/500 вакансий
Обработано 140/500 вакансий
Обработано 150/500 вакансий
Обработано 160/500 вакансий
Обработано 170/500 вакансий
Обработано 180/500 вакансий
Обработано 190/500 вакансий
Обработано 200/500 ваканс

In [26]:
import ast

def process_dataset(df):
    df_processed = df.copy()
    
    def format_salary(salary):
        if pd.isna(salary) or salary == '' or salary == 'None':
            return 'Не указана'
        
        # Если salary это строка, попробуем преобразовать в словарь
        if isinstance(salary, str):
            try:
                salary = ast.literal_eval(salary)
            except:
                return 'Не указана'
        
        if isinstance(salary, dict):
            parts = []
            if salary.get('from'):
                parts.append(f"от {salary['from']}")
            if salary.get('to'):
                parts.append(f"до {salary['to']}")
            
            if not parts:
                return 'Не указана'
            
            currency = salary.get('currency', '')
            currency_map = {
                'RUR': 'руб.',
                'RUB': 'руб.', 
                'USD': '$',
                'EUR': '€',
                'KZT': 'тенге'
            }
            
            currency_str = currency_map.get(currency, currency)
            result = ' '.join(parts)
            if currency_str:
                result += f' {currency_str}'
            
            gross = salary.get('gross')
            if gross is not None:
                result += ' (до вычета налогов)' if gross else ' (на руки)'
            
            return result
        else:
            return 'Не указана'
    
    df_processed['salary'] = df_processed['salary'].apply(format_salary)
    
    # Обработка столбца key_skills
    def format_skills(skills):
        if pd.isna(skills) or skills == '' or skills == '[]':
            return ''
        
        if isinstance(skills, str):
            try:
                # Пробуем преобразовать строку в список
                skills_list = ast.literal_eval(skills)
                if isinstance(skills_list, list):
                    return ', '.join(skills_list)
            except:
                # Если не получается, очищаем строку
                skills = re.sub(r'[\[\]\'\"]', '', skills)
                return skills
        elif isinstance(skills, list):
            return ', '.join(skills)
        else:
            return str(skills)
    
    df_processed['key_skills'] = df_processed['key_skills'].apply(format_skills)
    
    # Обработка столбца professional_roles
    def format_roles(roles):
        if pd.isna(roles) or roles == '' or roles == '[]':
            return ''
        
        if isinstance(roles, str):
            try:
                roles_list = ast.literal_eval(roles)
                if isinstance(roles_list, list):
                    return ', '.join(roles_list)
            except:
                roles = re.sub(r'[\[\]\'\"]', '', roles)
                return roles
        elif isinstance(roles, list):
            return ', '.join(roles)
        else:
            return str(roles)
    
    df_processed['professional_roles'] = df_processed['professional_roles'].apply(format_roles)
    
    # Обработка даты - оставляем только дату без времени
    def format_date(date_str):
        if pd.isna(date_str) or date_str == '':
            return ''
        
        if isinstance(date_str, str):
            return date_str[:10] if len(date_str) >= 10 else date_str
        else:
            return str(date_str)
    
    df_processed['published_at'] = df_processed['published_at'].apply(format_date)
    
    # Очистка описания от лишних пробелов
    def clean_description(desc):
        if pd.isna(desc) or desc == '':
            return ''
        
        # Заменяем множественные пробелы и переносы на один пробел
        cleaned = re.sub(r'\s+', ' ', str(desc)).strip()
        return cleaned
    
    df_processed['description'] = df_processed['description'].apply(clean_description)
    
    # Переименовываем столбцы
    column_rename = {
        'id': 'ID',
        'name': 'Название_вакансии',
        'salary': 'Зарплата',
        'area': 'Город',
        'published_at': 'Дата_публикации',
        'employer': 'Работодатель',
        'experience': 'Опыт',
        'employment': 'Тип_занятости',
        'description': 'Описание',
        'key_skills': 'Ключевые_навыки',
        'schedule': 'График',
        'professional_roles': 'Профессиональные_роли',
        'url': 'Ссылка'
    }
    
    df_processed = df_processed.rename(columns=column_rename)
    
    # Упорядочиваем столбцы
    column_order = [
        'ID', 'Название_вакансии', 'Зарплата', 'Город', 'Дата_публикации',
        'Работодатель', 'Опыт', 'Тип_занятости', 'Описание', 'Ключевые_навыки',
        'График', 'Профессиональные_роли', 'Ссылка'
    ]
    
    existing_columns = [col for col in column_order if col in df_processed.columns]
    df_processed = df_processed[existing_columns]
    
    return df_processed

def save_processed_dataset(df, filename=None):

    data_folder = "data"
    if not os.path.exists(data_folder):
        os.makedirs(data_folder)
    
    if filename is None:
        filename = f"processed_hh_vacancies_{datetime.now().strftime('%Y-%m-%d_%H-%M')}.csv"
    else:
        filename = os.path.join(data_folder, filename)
    
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Обработанный датасет сохранен в {filename}")
    return filename

if __name__ == "__main__":
    original_file = "data/hh_vacancies_2025-10-12_22-25.csv" 
    df_original = pd.read_csv(original_file, encoding='utf-8-sig')
    
    df_processed = process_dataset(df_original)
    
    save_processed_dataset(df_processed)
    
    print(f"\nРазмер датасета: {df_processed.shape}")
    print(f"\nСтолбцы: {list(df_processed.columns)}")

Обработанный датасет сохранен в processed_hh_vacancies_2025-10-12_22-25.csv

Размер датасета: (500, 13)

Столбцы: ['ID', 'Название_вакансии', 'Зарплата', 'Город', 'Дата_публикации', 'Работодатель', 'Опыт', 'Тип_занятости', 'Описание', 'Ключевые_навыки', 'График', 'Профессиональные_роли', 'Ссылка']


In [4]:
df_original = pd.read_csv(r'C:\Users\Zver\Desktop\machine_learning\notebook\nlp\data\processed_hh_vacancies_2025-10-12_22-25.csv', encoding='utf-8-sig')

In [6]:
df_original.sample(5)

,ID,Название_вакансии,Зарплата,Город,Дата_публикации,Работодатель,Опыт,Тип_занятости,Описание,Ключевые_навыки,График,Профессиональные_роли,Ссылка
149,125797603,Middle Backend разработчик (R&D / Startup-прое...,Не указана,Москва,2025-10-10,Aiti Guru,От 1 года до 3 лет,Полная занятость,В связи с ростом компании и запуском новых про...,"JavaScript, Python, REST, SQL, Git, HTML, Разр...",Полный день,"Программист, разработчик",https://hh.ru/vacancy/125797603
243,125158150,Data Scientist/ ML Engineer в направление LLM,Не указана,Москва,2025-10-10,X5 Tech,От 3 до 6 лет,Полная занятость,X5 Group — российская розничная торговая компа...,"Python, Машинное обучение",Полный день,Дата-сайентист,https://hh.ru/vacancy/125158150
84,124880155,Head of Python Development,Не указана,Москва,2025-10-12,Хантфлоу,Более 6 лет,Полная занятость,Хантфлоу — главный инструмент работы рекрутеро...,"Python, FastAPI, PostgreSQL, Tornado, TechLead",Удаленная работа,"Программист, разработчик",https://hh.ru/vacancy/124880155
102,123761902,Разработчик Python (Серверная Астра),Не указана,Москва,2025-10-06,Группа компаний Астра,От 3 до 6 лет,Полная занятость,Мы группа компаний «Астра» – один из лидеров р...,"Linux, Python",Удаленная работа,"Программист, разработчик",https://hh.ru/vacancy/123761902
206,126174282,Риск-менеджер / Инвестиционный аналитик (Junio...,до 100000 руб. (на руки),Москва,2025-10-12,ADF Capital,От 1 года до 3 лет,Полная занятость,Привет! Мы ADF Capital — инновационная проп-тр...,"Анализ рисков, Аналитическое мышление, SQL, Оц...",Полный день,Финансовый менеджер,https://hh.ru/vacancy/126174282
